In [1]:
import numpy as np
import pandas as pd
import os 
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset,DataLoader
from torchmetrics.classification import BinaryAUROC 
import re
from sklearn.preprocessing import StandardScaler,QuantileTransformer
from sklearn.model_selection import train_test_split
train_final=pd.read_csv("D:/Desktop/机器学习的数据集/Home Credit Default Risk/train_final_without_ohe.csv")
test_final=pd.read_csv("D:/Desktop/机器学习的数据集/Home Credit Default Risk/test_final_without_ohe.csv")   

In [2]:
train_final.columns = [re.sub(r"[^a-zA-Z0-9_\-]","_",str(col)) for col in train_final.columns]
test_final.columns = [re.sub(r"[^a-zA-Z0-9_\-]","_",str(col)) for col in test_final.columns]
train_final.drop("Unnamed__0",axis=1,inplace=True)
test_final.drop("Unnamed__0",axis=1,inplace=True)

In [3]:
train_final

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,...,AMT_CREDIT_SUM_DEBT_SUM_mean,AMT_CREDIT_SUM_DEBT_SUM_max,AMT_CREDIT_SUM_DEBT_SUM_sum,CREDIT_ACTIVE_count,CREDIT_ACTIVE_nunique,CREDIT_CURRENCY_count,CREDIT_CURRENCY_nunique,CREDIT_TYPE_count,CREDIT_TYPE_nunique,Bureau_flag
0,100002,Cash loans,M and Others (new),N,Y,202500.0,406597.5,24700.5,351000.0,Unaccompanied,...,34721.195625,245781.0,277769.565,8.0,2.0,8.0,1.0,8.0,2.0,1
1,100003,Cash loans,F,N,N,270000.0,1293502.5,35698.5,1129500.0,Family,...,202500.000000,810000.0,810000.000,4.0,2.0,4.0,1.0,4.0,2.0,1
2,100004,Revolving loans,M and Others (new),Y,Y,67500.0,135000.0,6750.0,135000.0,Unaccompanied,...,0.000000,0.0,0.000,2.0,1.0,2.0,1.0,2.0,1.0,1
3,100006,Cash loans,F,N,Y,135000.0,312682.5,29686.5,297000.0,Unaccompanied,...,0.000000,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0
4,100007,Cash loans,M and Others (new),N,Y,121500.0,513000.0,21865.5,513000.0,Unaccompanied,...,0.000000,0.0,0.000,1.0,1.0,1.0,1.0,1.0,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,Cash loans,M and Others (new),N,N,157500.0,254700.0,27558.0,225000.0,Unaccompanied,...,0.000000,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0
307507,456252,Cash loans,F,N,Y,72000.0,269550.0,12001.5,225000.0,Unaccompanied,...,0.000000,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0
307508,456253,Cash loans,F,N,Y,153000.0,677664.0,29979.0,585000.0,Unaccompanied,...,448958.250000,1624797.0,1795833.000,4.0,2.0,4.0,1.0,4.0,2.0,1
307509,456254,Cash loans,F,N,Y,171000.0,370107.0,20205.0,319500.0,Unaccompanied,...,0.000000,0.0,0.000,1.0,1.0,1.0,1.0,1.0,1.0,1


In [4]:
train_final.fillna(0,inplace=True)
df = train_final.copy()

# 分离数值列和类别列..简单的通过这里的判断类型得到的分类特征还是太少，所以这时候还是需要进行筛选
categorical_cols = df.select_dtypes(include=['object',"O"]).columns.tolist()
numerical_cols = [c  for c in df.columns if c not in categorical_cols]

# 移除 ID 列（如 SK_ID_CURR）
if 'SK_ID_CURR' in numerical_cols:
    numerical_cols.remove('SK_ID_CURR')
nunique_counts = train_final.nunique()
# 筛选出唯一值数量 < 20 的列
low_cardinality = nunique_counts[nunique_counts < 60][1:]
# 转为 DataFrame 并命名列
count_df = low_cardinality.to_frame(name='NUMBER_COLUMNS')
length=[]
for col  in count_df.index:
    if col not in categorical_cols:
        length.append(col)
len(length)

categorical_cols=[c for c in length if c!="TARGET"]
numerical_cols=[c for c in numerical_cols if c not in categorical_cols]

In [5]:
print(len(categorical_cols))
len(numerical_cols)

99


367

In [6]:
target=train_final.TARGET
ids = test_final.SK_ID_CURR
#print(test_final.head())
train=df.drop(["SK_ID_CURR","TARGET"],axis=1)
test_final.drop(["SK_ID_CURR"],axis=1,inplace=True)
cols=["TARGET",'SK_ID_PREV','SK_ID_BUREAU','SK_ID_CURR']
numerical_cols=[col for col in numerical_cols if col not in cols]

In [7]:
x_train,x_test,y_train,y_test=train_test_split(train,target,test_size = 0.10, random_state = 1008, stratify=target)

In [8]:
# 健壮性的标签编码格式
class RobustLabelEncoder:
    def __init__(self,min_obs=1):
        self.min_obs=min_obs
        self.mapper={}
        self.cate_dims = 0
    def fit(self,series):
        value_counts=series.value_counts()
        valid_cols=value_counts[value_counts>self.min_obs].index.tolist()
        # 构建映射表,从1开始映射
        self.mapper={val:i+1 for i,val in enumerate(valid_cols)}
        self.cate_dims=len(self.mapper)+1       # 记录类型标签的长度
        return self
    def transform(self,series):
        return series.map(self.mapper).fillna(0).astype(int).values   # 对于在fit中没有出现的类型，填充为0，这也就是为什么上面的fit是从i+1开始的
    def fit_transform(self,series):
        self.fit(series)
        return self.transform(series)

In [9]:
# 这是对数值性特征进行的处理，也就是说不管是什么形式的分布，尤其是涉及到钱的右偏分布，都适合把这里变成正态分布。
scaler=QuantileTransformer(n_quantiles=2000,                 # 采样的份数，越大越精确，但容易过拟合。对于大这类数据，设为 2000 或 len(train) 都可以
                           output_distribution='normal',    # 变成高斯分布的关键
                           random_state=108)
x_train_numerical=x_train[numerical_cols].copy()
x_test_numerical=x_test[numerical_cols].copy()
train_new_cols={}
test_new_cols={}
for col in numerical_cols:   # 防止某些数值性字段出现无穷大的，就没有办法进行标签编码
    x_train_numerical[col] = x_train_numerical[col].replace([np.inf, -np.inf], np.nan)
    x_test_numerical[col] = x_test_numerical[col].replace([np.inf, -np.inf], np.nan)
    if x_train_numerical[col].isnull().any():    #如果一列之中有缺失值存在
         # 创建一个新列标注是否缺失.先使用一个字典存储好新的列，这样不用反复的调用本来很大的的x_train_numerical
         train_new_cols[f"{col}_is_NaN"]=x_train_numerical[col].isnull().astype(int)  # astype()可以把bool值变成数字1，0
         test_new_cols[f"{col}_is_NaN"]=x_test_numerical[col].isnull().astype(int) 
         # 此时对缺失值进行填充。用中位数填充会忽略到不填收入的人的违约风险，所以在填充前进行人员的标注
         med = x_train_numerical[col].median()
         if pd.isna(med):
             med=0
         x_train_numerical[col] = x_train_numerical[col].fillna(med)
         x_test_numerical[col] = x_test_numerical[col].fillna(med)
x_train_numerical =pd.concat([x_train_numerical,pd.DataFrame(train_new_cols)],axis=1)
numerical_cols=x_train_numerical.columns.to_list()
x_test_numerical  =pd.concat([x_test_numerical,pd.DataFrame(test_new_cols)],axis=1)
x_train_numerical = scaler.fit_transform(x_train_numerical)  # 返回的是numpy的格式
x_test_numerical  = scaler.transform(x_test_numerical)


# 对非数值性特征进行处理
cate_dims_ls=[]
valid_categorical_cols = [] 
x_train_cate_encoded = x_train[categorical_cols].copy()
x_test_cate_encoded = x_test[categorical_cols].copy()
for col in categorical_cols:
     x_train_cate_encoded[col]=x_train_cate_encoded[col].fillna("MISSING_VAL").astype(str)
     x_test_cate_encoded[col]=x_test_cate_encoded[col].fillna("MISSING_VAL").astype(str)
     encoder=RobustLabelEncoder(min_obs=1)
     encoder.fit(x_train_cate_encoded[col])
    #在训练集和测试集上 TRANSFORM
    # 关键点：如果 x_test 里出现了 x_train 里没有的类别，会自动变为 0
     x_train_cate_encoded[col] = encoder.transform(x_train_cate_encoded[col])
     x_test_cate_encoded[col] = encoder.transform(x_test_cate_encoded[col])
     max_index_train =x_train_cate_encoded[col].max()
     max_index_test=x_test_cate_encoded[col].max()
     num_embeddings = max(int(max_index_train),int(max_index_test)) + 1
     #print(f"{col}_dim_{num_embeddings}")
     if num_embeddings>100 or num_embeddings==1:
          continue
     cate_dims_ls.append(num_embeddings)
     valid_categorical_cols.append(col)
print("分类特征维度:", cate_dims_ls)
print("分类特征数量:", len(cate_dims_ls))

# 设置每个类别特征的嵌入维度（这是一个超参数）
# 经验法则: emb_dim = min(50, (cardinality + 2) // 2)
emb_dims = [(x, min(50, (x + 2) // 2)) for x in cate_dims_ls]

分类特征维度: [3, 3, 3, 3, 15, 4, 25, 3, 3, 3, 30, 8, 29, 8, 3, 3, 3, 5, 9, 10, 21, 10, 21, 3, 6, 5, 26, 21, 2, 2, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 6, 48, 30, 13, 33, 4, 4, 4, 4, 5, 18, 3, 5, 32, 3, 5, 9, 9, 3, 8, 3, 44, 4, 4, 4, 3]
分类特征数量: 99


In [10]:
x_train_numerical

array([[ 1.58460315,  1.31029799,  0.68195677, ..., -5.19933758,
        -5.19933758, -5.19933758],
       [-0.18602365,  1.13201337,  0.51750683, ..., -5.19933758,
        -5.19933758, -5.19933758],
       [ 0.67330956, -0.76005182, -0.27130367, ..., -5.19933758,
        -5.19933758, -5.19933758],
       ...,
       [ 0.14217286,  1.33129627,  0.7112196 , ..., -5.19933758,
        -5.19933758, -5.19933758],
       [-0.18602365,  0.48345357,  0.27897087, ..., -5.19933758,
        -5.19933758, -5.19933758],
       [-1.29130396, -1.87327431, -1.07491097, ..., -5.19933758,
        -5.19933758, -5.19933758]], shape=(276759, 369))

In [16]:
class SwapNoise(nn.Module):
    def __init__(self, probability=0.15):  # 这里的0.15是说Batch
        super().__init__()
        self.probability = probability

    def forward(self, x):
        if not self.training:
            return x
        # 生成掩码：1代表保持，0代表交换
        # torch.full_like(x, 1 - self.probability)生成了和x形状完全一样的全为0.85的张量
        #mask = torch.bernoulli(torch.full_like(x, 1 - self.probability))
        mask =torch.rand_like(x.float()) > self.probability
        
        # 在 Batch 内部随机打乱样本顺序
        batch_size = x.size(0)
        # torch.randperm(n)`：返回 `[0, 1, ..., n-1]` 的一个随机排列
        idx = torch.randperm(batch_size, device=x.device)
        shuffled_x = x[idx]
        
        # 混合：原数据 * 掩码 + 打乱数据 * (1-掩码)
        #return x * mask + shuffled_x * (1 - mask)
        return torch.where(mask,x,shuffled_x)

class Tabular_Transformer(nn.Module):
    def __init__(self, num_numerical, cate_dims_ls, dropout=0.10,num_blocks=3,embed_dim=6):
        super().__init__()
        self.swap_noise = SwapNoise(probability=0.15)
        self.embed_dim=embed_dim
        self.cate_embedding_layers = nn.ModuleList([
            nn.Embedding(num, embed_dim) for num in cate_dims_ls
        ])

        #为每个数值特征准备一个专用的 Linear 层
        self.num_projections = nn.ModuleList([
            nn.Linear(1, embed_dim) for _ in range(num_numerical)
        ])

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embed_dim, 
            nhead=8,            # 8头注意力
            dim_feedforward=embed_dim * 2,   # 前馈网络（FFN）中间层的维度
            dropout=dropout, 
            activation='gelu',  # 前馈网络中的激活函数 
            batch_first=True    # 输入张量的格式：True: (batch_size, seq_len, d_model).。。是为了和ResNret好链接
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_blocks)
        self.cls_token=nn.Parameter(torch.zeros(1,1,embed_dim))

        # --- Decoder (只用于预训练) ---
        # 目标是还原回 total_dim 维度的原始输入
        self.num_decoder = nn.Sequential(
            nn.Linear(embed_dim, embed_dim),
            nn.LayerNorm(embed_dim),
            nn.ReLU(),
            nn.Linear(embed_dim,1) 
        )
        self.cate_decoders=nn.ModuleList()
        for num_cate in cate_dims_ls:
            cate_decoder=nn.Sequential(
            nn.Linear(embed_dim, embed_dim),
            nn.ReLU(),
            nn.Linear(embed_dim,num_cate) )
            self.cate_decoders.append(cate_decoder)

        # 用于最终分类的过程
        self.classifier_head = nn.Sequential(
            nn.Linear(embed_dim, embed_dim),
            nn.LayerNorm(embed_dim),
            nn.PReLU(),
            nn.Linear(embed_dim, 1)
        )

    def embed_total_dims(self,x_num,x_cate):
        x_num_embed=[]
        for i,proj in enumerate(self.num_projections):
            col=x_num[:,i].unsqueeze(1)  # (Batch_size,1)
            proj
            x_num_embed.append(proj(col)) # (Batch_size,num_features)
        x_num_embed=torch.stack(x_num_embed,dim=1)  # torch.stack()会沿新维度 dim=1 堆叠这 N 个 (B, d) 张量。  （B，num_features,embed）
        #print(x_num_embed.shape)

        x_cate_embed=[]
        for i,proj in enumerate(self.cate_embedding_layers):
            col=x_cate[:,i].unsqueeze(1)   # (Batch_size,1)
            proj_col=proj(col).squeeze(1)  # proj(col)是（batch_SIZE,1，embed_dim）,所以需要把它先压扁
            #print(proj(col).shape)
            x_cate_embed.append(proj_col) # (Batch_size,num_features)
        x_cate_embed=torch.stack(x_cate_embed,dim=1)  # torch.stack()会沿新维度 dim=1 堆叠这 N 个 (B, d) 张量。  （B，num_features,embed）
        # print(x_cate.shape)        # ([1024,102])
        # print(x_cate_embed.shape)  # ([1024, 102, 1, 32])

        x=torch.cat([x_num_embed,x_cate_embed],dim=1)
        return x
    def forward_pretrain(self,x_cate,x_num):
        x_cate_noisy = self.swap_noise(x_cate)
        x_num_noisy = self.swap_noise(x_num)
        x=self.embed_total_dims(x_num=x_num_noisy,x_cate=x_cate_noisy)

        cls_token=self.cls_token.expand(x.shape[0],-1,-1)
        x=torch.cat([cls_token,x],dim=1)
        x = self.transformer(x)
        x_features=x[:,1:,:]     # 我们不需要 CLS Token 的输出来做重建，只取后面的特征部分

        # 拆分为数值特征和类别特征
        n_num=x_num_noisy.shape[1]
        x_num_out = x_features[:, :n_num, :] # (Batch, N_num, embed_dim)
        x_cat_out = x_features[:, n_num:, :] # (Batch, N_cat, embed_dims)

       
        pred_num_list = self.num_decoder(x_num_out).squeeze(-1) # (Batch, N_num)
        #print(pred_num_list)
        
        # B. 预测类别特征 (分类)
        pred_cat_list = []
        # for i, decoder in enumerate(self.cate_decoders):
        #     # 输入第 i 个类别特征的 embedding，预测类别 logits
        #     pred_logits = decoder(x_cat_out[:, i, :]) # (Batch, Num_Classes)
        #     pred_cat_list.append(pred_logits)
        return pred_num_list
        

    def forward_classify(self,x_cate,x_num):
        x=self.embed_total_dims(x_num,x_cate)
        cls_token=self.cls_token.expand(x.shape[0],-1,-1)
        x=torch.cat([cls_token,x],dim=1)
        x = self.transformer(x)
        cls_output=x[:,0,:]     # 这个其实就是cls_token汇总的信息量
        logitis=self.classifier_head(cls_output)

        return logitis

In [12]:
x_train_numerical_df=pd.DataFrame(scaler.fit_transform(x_train_numerical),columns=numerical_cols)
x_test_numerical_df =pd.DataFrame(scaler.transform(x_test_numerical),columns=numerical_cols)

In [13]:
num_numerical_features=len(numerical_cols)
print(y_train)
if isinstance(y_train,np.ndarray):
    y_train=pd.Series(y_train.flatten())
y_train = y_train.values.astype('float32').reshape(-1, 1)  # 用于 BCE loss..这样y_train就是[n_samples,1]
X_train_num_tensor = torch.tensor(x_train_numerical, dtype=torch.float32)
X_train_cate_tensor = torch.tensor(x_train_cate_encoded[valid_categorical_cols].values, dtype=torch.long)     # 嵌入层的输入对于分类特征必须是torch.long的格式
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
train_dataset=TensorDataset(X_train_num_tensor,X_train_cate_tensor,y_train_tensor)         
train_loader=DataLoader(train_dataset,batch_size=128,shuffle=True)

if isinstance(y_test,np.ndarray):
    y_test=pd.Series(y_test.flatten())
y_test = y_test.values.astype('float32').reshape(-1, 1)  # 用于 BCE loss
X_test_num_tensor = torch.tensor(x_test_numerical, dtype=torch.float32)
X_test_cate_tensor= torch.tensor(x_test_cate_encoded[valid_categorical_cols].values,dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)
test_dataset=TensorDataset(X_test_num_tensor,X_test_cate_tensor,y_test_tensor)         
test_loader=DataLoader(test_dataset,batch_size=128,shuffle=False)

device=torch.device("cuda" if torch.cuda.is_available() else "cpu")

213080    0
224440    0
148283    0
56155     0
7812      0
         ..
3461      0
4792      0
101428    0
126394    0
114578    0
Name: TARGET, Length: 276759, dtype: int64


In [ ]:
# 初始化模型
num_numerical_features=len(numerical_cols)
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
model=Tabular_Transformer(num_numerical=num_numerical_features,embed_dim=32,cate_dims_ls=cate_dims_ls,num_blocks=3,dropout=0.15)
#model = DAE_MLP_Network(num_numerical=num_numerical_features, emb_dims=emb_dims).to(device)
# 专门的优化器
pretrain_optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-3)
# 数值特征用 HuberLoss 或 SmoothL1Loss (抗异常值)。。实质函数是MSE的进阶版本
criterion_num = nn.HuberLoss(delta=1.0) 
# 类别特征用 CrossEntropyLoss   （交叉熵损失）
criterion_cate = nn.CrossEntropyLoss()
print("🚀 开始 DAE 预训练...")
EPOCHS_PRETRAIN = 10
loss_history=[]
for epoch in range(EPOCHS_PRETRAIN):
    model.train()
    total_loss = 0
    
    # 遍历 DataLoader (这里的 y 在预训练中没用)
    for x_num, x_cate, _ in train_loader:
        x_num, x_cate = x_num.to(device), x_cate.to(device)    
        # 前向传播 (预训练模式)
        # 输出：类别特征的预测，数值特征的预测
        pred_num = model.forward_pretrain(x_cate, x_num)
        # 计算 Loss：模型学到的特征和真实特征之间的差值
        loss_num = criterion_num(pred_num,x_num)
        loss_cate=0
        # 忽略对类别特征的计算
        # for i,pred_cate in enumerate(pred_cates):
        #     #print(pred_cate.shape)  
        #     #print(x_cate.shape)     # （1024，102）
        #     loss_cate +=criterion_cate(pred_cate,x_cate[:,i])
        loss=loss_num     #+(loss_cate/len(pred_cates))
        
        pretrain_optimizer.zero_grad()
        loss.backward()
        pretrain_optimizer.step()
        total_loss += loss.item()
    avg_loss = total_loss / len(train_loader)
    loss_history.append(avg_loss)
    print(f"Pretrain Epoch {epoch+1}/{EPOCHS_PRETRAIN} | MSE Loss: {avg_loss:.6f}")
torch.save(model.state_dict(),"pretrain_dae.pth")
print("✅ DAE 预训练完成！模型已经学会了数据的内部结构。")

🚀 开始 DAE 预训练...
Pretrain Epoch 1/10 | MSE Loss: 0.537340
Pretrain Epoch 2/10 | MSE Loss: 0.181879
Pretrain Epoch 3/10 | MSE Loss: 0.168756


In [ ]:
def train_each_epoch(model,train_loader,device,loss,optimizer,auc_metric):
    metrics = {
        'loss': 0.0,
        'samples': 0,
    }
    model.train()         # 切换为训练模式
    # 在每个 epoch 开始时，重置 metric 的内部状态
    auc_metric.reset()
    for X_num,X_cate,y in train_loader:
        X_num,X_cate, y = X_num.to(device), X_cate.to(device),y.to(device)
        y_hat,_=model.forward_classify(X_cate,X_num)
        l=loss(y_hat,y)   # 因为nn.BCEWithLogitsLoss()默认返回的是平均损失，所以要算总损失还要用  l*样本数

        # 更新过程
        optimizer.zero_grad()
        l.backward()
        optimizer.step() #更新权重
        
        batch_size=y.numel()                        # 也可以使用y.size(0)
        metrics["loss"] += l.item()*batch_size      # l.item()返回的是一个python的数值
        metrics["samples"] +=y.numel()

         # (新增) 更新AUC metric
        # .update() 会累计每个批次的结果
        # 注意：torchmetrics 需要整数标签，所以用 .int()
        auc_metric.update(y_hat,y.int())
    metrics["auc"]  = auc_metric.compute().item()
    return metrics
def eval_each_epoch(model,test_loader,device,loss,auc_metric):
    metrics = {
        'loss': 0.0,
        'samples': 0
    }
    model.eval()       # 切换为评估模式
    auc_metric.reset()
    with torch.no_grad():
        for X_num,X_cate,y in test_loader:
            X_num,X_cate, y = X_num.to(device), X_cate.to(device),y.to(device)
            y_hat,latent_val=model.forward_classify(X_cate,X_num)
            l=loss(y_hat,y)   # 因为nn.BCEWithLogitsLoss()默认返回的是平均损失，所以要算总损失还要用  l*样本数
            batch_size=y.size(0)
            metrics["loss"] += l.item()*batch_size      # l.item()返回的是一个python的数值
            metrics["samples"] +=y.numel()
            auc_metric.update(y_hat,y.int())
    metrics["auc"]  = auc_metric.compute().item()
    return metrics


def train_epoches(model,train_loader,test_loader,device,loss,optimizer,epoches=20):
    train_auc_metric = BinaryAUROC().to(device)
    val_auc_metric = BinaryAUROC().to(device)
    patience = 5 
    patience_counter=0
    best_val_auc=float('-inf')
    eval_auc_epoches=[]
    for epoch in range(epoches):
        train_metrics = train_each_epoch(model,train_loader,device,loss,optimizer,auc_metric=train_auc_metric)
        eval_metrics  = eval_each_epoch(model,test_loader,device,loss,auc_metric=val_auc_metric)
        avg_loss=train_metrics['loss'] / train_metrics['samples']
        train_auc=train_metrics["auc"]

        eval_avg_loss = eval_metrics['loss'] / eval_metrics['samples']
        eval_auc=eval_metrics["auc"]
        eval_auc_epoches.append(eval_auc)

        if eval_auc>best_val_auc:
            best_val_auc=eval_auc
            print(f"保存的是第{epoch}轮的模型")
            torch.save(model.state_dict(),'best_model.pth')
            print(f"Epoch {epoch}: New best model saved with Val AUC: {eval_auc:.4f},,,,,{best_val_auc:.4f}")
            patience_counter = 0 # 重置耐心计数器
        else:
            patience_counter +=1
        if patience_counter >=patience:
            print(f"模型在第{epoch-5}轮中接下来持续{patience}轮无法做到AUC更好，停止模型的训练")
            #break
    return model,eval_auc_epoches
# 重新初始化优化器 (这是微调的关键，学习率通常比预训练小一点，或者结合 OneCycleLR)
#finetune_optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
# 分类损失函数
#criterion = nn.BCEWithLogitsLoss() 
#train_epoches(model=model,train_loader=train_loader,test_loader=test_loader,device=device,loss=criterion,optimizer=finetune_optimizer,epoches=20)

In [ ]:
# 下面进行5折分层神经网络训练
from sklearn.model_selection import StratifiedKFold
kf=StratifiedKFold(n_splits=5,random_state=108,shuffle=True)
nn_folds=list(kf.split(x_train_numerical,y_train))

def train_NN_oof(x_train_numerical, x_train_cate_encoded, y_train,x_test_numerical,x_test_cate_encoded, y_test,folds):
    if isinstance(y_train,pd.Series):
        y_train=y_train.values
    oof = np.zeros(len(y_train))
    oof_latents = np.zeros((len(y_train), 512), dtype=np.float32)
    # 用于测试集的部分
    test_latents_all=np.zeros((len(x_test_numerical), 512), dtype=np.float32)
    preds_test = np.zeros(len(x_test_numerical))
     # 在最佳模型预测下进行
    y_test = y_test.astype('float32').reshape(-1, 1)  # 用于 BCE loss
    X_test_num_tensor = torch.tensor(x_test_numerical, dtype=torch.float32)
    X_test_cate_tensor= torch.tensor(x_test_cate_encoded[valid_categorical_cols].values,dtype=torch.long)
    y_test_tensor = torch.tensor(y_test, dtype=torch.float32)
    test_dataset=TensorDataset(X_test_num_tensor,X_test_cate_tensor,y_test_tensor)         
    test_loader=DataLoader(test_dataset,batch_size=1024,shuffle=False)

    for fold, (train_idx, valid_idx) in enumerate(folds):
        print(f"=== NN Fold {fold+1} ===")
        print(valid_idx.max()) 
        y_train_idx = y_train[train_idx].astype('float32').reshape(-1, 1)                # 用于 BCE loss..这样y_train就是[n_samples,1]
        X_train_num_tensor_idx = torch.tensor(x_train_numerical[train_idx], dtype=torch.float32)
        X_train_cate_tensor_idx = torch.tensor(x_train_cate_encoded[valid_categorical_cols].values[train_idx], dtype=torch.long)     # 嵌入层的输入对于分类特征必须是torch.long的格式
        y_train_tensor_idx = torch.tensor(y_train_idx, dtype=torch.float32)
        train_dataset_idx=TensorDataset(X_train_num_tensor_idx,X_train_cate_tensor_idx,y_train_tensor_idx)         
        train_loader_idx=DataLoader(train_dataset_idx,batch_size=1024,shuffle=True)


        y_val_idx = y_train[valid_idx].astype('float32').reshape(-1, 1)  # 用于 BCE loss
        X_val_num_tensor_idx = torch.tensor(x_train_numerical[valid_idx], dtype=torch.float32)
        X_val_cate_tensor_idx= torch.tensor(x_train_cate_encoded[valid_categorical_cols].values[valid_idx],dtype=torch.long)
        y_val_tensor_idx = torch.tensor(y_val_idx, dtype=torch.float32)
        val_dataset_idx=TensorDataset(X_val_num_tensor_idx,X_val_cate_tensor_idx,y_val_tensor_idx)         
        val_loader_idx=DataLoader(val_dataset_idx,batch_size=1024,shuffle=False)
        
        # 分类损失函数,使用BCEWithLogitsLoss函数  用于二分类（binary classification） 任务的损失计算。本质是 signmod函数和交叉熵损失函数的融合
        criterion = nn.BCEWithLogitsLoss() 
        model.load_state_dict(torch.load('pretrain_dae.pth'), strict=False)

        # 使用预训练的过程进行初始化权重,这时候要把除了classifier_head的给锁住，也就是不能进行反向传播了
        for name,param in model.named_parameters():
            if "classifier_head" not in name:
                param.requires_grad=False
        first_optimizer=torch.optim.AdamW(filter(lambda x:x.requires_grad,model.parameters()),lr=1e-4)
        model.train()
        for X_num,X_cate,y in train_loader_idx:
            X_num,X_cate, y = X_num.to(device), X_cate.to(device),y.to(device)
            y_hat,_=model.forward_classify(X_cate,X_num)
            l=criterion(y_hat,y)   # 因为nn.BCEWithLogitsLoss()默认返回的是平均损失，所以要算总损失还要用  l*样本数
            # 更新过程
            first_optimizer.zero_grad()
            l.backward()
            first_optimizer.step() #更新权重

        #  这时候再正式开始训练，评估
        for param in model.parameters():
            param.requires_grad = True
        finetune_optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
        best_model,eval_auc_epoches=train_epoches(model=model,train_loader=train_loader_idx,test_loader=val_loader_idx,device=device,loss=criterion,optimizer=finetune_optimizer,epoches=7)
        print("在验证集上的AUC变化过程")
                # 设置画布大小
        plt.figure(figsize=(10, 6))
        plt.plot(range(1, 8),eval_auc_epoches , marker='o', linestyle='-', color='b', label='EVal AUC 7-Epoches')
        # 添加标题和标签
        plt.title('在验证集上的AUC变化过程', fontsize=16)
        plt.xlabel('Epochs', fontsize=12)
        plt.ylabel('AUC', fontsize=12)
        plt.grid(True, linestyle='--', alpha=0.7)
        plt.legend()
        plt.show()


        best_model.eval()
        test_preds_fold=[]
        test_latents_fold=[]
        with torch.no_grad():
            # 验证集
            best_model.load_state_dict(torch.load("best_model.pth"),strict=False)
            best_preds,latent_val=best_model.forward_classify(X_val_cate_tensor_idx,X_val_num_tensor_idx)
            oof[valid_idx]=torch.sigmoid(best_preds).cpu().numpy().flatten()
            oof_latents[valid_idx] = latent_val.cpu().numpy()
            oof_latents_df=pd.DataFrame(oof_latents,columns=[f"latent_{i}" for i in range(oof_latents.shape[1])])
            # 测试集（分batch处理）
            for X_num,X_cate,y in test_loader:
                X_num,X_cate, y = X_num.to(device), X_cate.to(device),y.to(device)
                y_hat,latent_test=best_model.forward_classify(X_cate,X_num)
                test_preds_fold.append(torch.sigmoid(y_hat).cpu().numpy().flatten())  # flatten（）是用于把二维数组展平的操作  
                test_latents_fold.append(latent_test.cpu().numpy())
        test_latents_fold=np.vstack(test_latents_fold)
        test_preds_fold = np.concatenate(test_preds_fold)
        preds_test += test_preds_fold/5   # 需要对numpy才可以这样直接除以5
        test_latents_all +=test_latents_fold/5
        test_latents_df=pd.DataFrame(test_latents_all,columns=[f"latent_{i}" for i in range(test_latents_all.shape[1])])
    return oof,preds_test,oof_latents,test_latents_all

nn_oof,nn_preds_test,latents_val,latents_test=train_NN_oof(x_train_numerical, x_train_cate_encoded, y_train,x_test_numerical,x_test_cate_encoded,y_test, nn_folds)
